In [146]:
import polars as pl
from Levenshtein import ratio

Constants

In [147]:
RATIO = .7 # similarity between T&Cs and gateway list
PARTIAL_RATIO = .5

In [148]:
terms = (
    pl.scan_csv("data/terms.csv", infer_schema_length=10000)
    
)

In [149]:
gateway = (
    pl.scan_csv('data/gateway_connections.csv', infer_schema_length=10000)
    .with_columns(
        pl.col('Licensee').str.replace_all(r'\([^)]*\)', '').alias('Licnesee_split')
        #strip_chars().str.split('(AZ)').alias('stripped_licensee')
    )
)


In [150]:
gateway.collect().write_excel(
    'data/test.xlsx',
    worksheet='match',
    autofit=True,
    freeze_panes=((1,0,0,0))
)